In [34]:
#librerias estandar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [35]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv", index_col="ID")
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


**Nota aclaratoria**

El preprocesado con umbral hace referencia a la estrategia de reducción de categorias de la variable PRGM_ACADEMICO. En este caso la reducción se hizo estableciendo un umbral que establece el porcentaje de programas que almacenarán en una misma categoria denominada "otros_programas" según la cantidad de estudiantes que pertenezcan al programa

In [36]:
#preprocesado

#tratamiento de variable objetivo
orden = ["bajo", "medio-bajo", "medio-alto", "alto"]
data['RENDIMIENTO_GLOBAL'] = pd.Categorical(data['RENDIMIENTO_GLOBAL'], ordered=True, categories=orden)
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].cat.codes  # convierte a 0,1,2,3

#Nulos
#True si el registro no tiene ningun valor nulo, False en caso contrario
data["DATOS_COMPLETOS"] = data.isna().sum(axis=1) == 0
data = data.fillna("sin_dato") #rellenar nulos

#columna repetida
data = data.drop(columns=["F_TIENEINTERNET.1"])

freq = data['E_PRGM_ACADEMICO'].value_counts()
acum = freq.cumsum() / freq.sum() # Cálculo de acumulado porcentual
umbral = freq[acum <= 0.90].min()
frecuencias = data['E_PRGM_ACADEMICO'].value_counts()
categorias_pequenas = frecuencias[frecuencias < umbral].index

data['E_PRGM_ACADEMICO_AGRUPADO'] = data['E_PRGM_ACADEMICO'].replace(categorias_pequenas, 'OTRO_PROGRAMA')
data.drop(columns=["E_PRGM_ACADEMICO"], inplace=True)

#binarias
binarias = [
    "F_TIENEINTERNET",
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
    "E_PRIVADO_LIBERTAD"
]

for col in binarias:
    data[col] = (
        data[col]
        .astype(str)
        .str.strip()
        .str.upper()
        .replace({
            "SI": 1, "S": 1,
            "NO": 0, "N": 0,
            "SIN_DATO": -1
        })
    )

#ordinales
# Codificación para E_VALORMATRICULAUNIVERSIDAD
map_matricula = {
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7,
    "No pagó matrícula": 0,
    "sin_dato": -1
}
data["E_VALORMATRICULAUNIVERSIDAD"] = data["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)

# Codificación para F_ESTRATOVIVIENDA
map_estrato = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": 0,
    "sin_dato": -1
}
data["F_ESTRATOVIVIENDA"] = data["F_ESTRATOVIVIENDA"].map(map_estrato)

# Codificación para E_HORASSEMANATRABAJA
map_horas = {
    "0": 0,   # por si aparece como string
    0: 0,     # por si aparece como número
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4,
    "sin_dato": -1
}
data["E_HORASSEMANATRABAJA"] = data["E_HORASSEMANATRABAJA"].map(map_horas)

educacion_map = {
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    
    # categorías sin información → -1
    "sin_dato": -1,
    "No sabe": -1,
    "No Aplica": -1
}

data["F_EDUCACIONPADRE"] = data["F_EDUCACIONPADRE"].map(educacion_map)
data["F_EDUCACIONMADRE"] = data["F_EDUCACIONMADRE"].map(educacion_map)

#one hot
data = pd.get_dummies(data, columns=["E_PRGM_DEPARTAMENTO"], prefix="DEP", drop_first=False)
data = pd.get_dummies(data, columns=["E_PRGM_ACADEMICO_AGRUPADO"], prefix="PRGM", drop_first=False)

#periodo academico
data["PERIODO_ACADEMICO"] = data["PERIODO_ACADEMICO"].astype(str)

data["ANIO"] = data["PERIODO_ACADEMICO"].str[:4].astype(int)
data["PERIODO"] = data["PERIODO_ACADEMICO"].str[-1].astype(int)

#numericas
cols_norm = ["INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]

# 3. Crear scaler y ajustarlo
scaler = StandardScaler()
data[cols_norm] = scaler.fit_transform(data[cols_norm])

#data.drop(columns=["E_PRGM_ACADEMICO_AGRUPADO"], inplace=True)

data.drop(columns=["PERIODO_ACADEMICO"], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_16520\1614207678.py:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_16520\1614207678.py:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({
C:\Users\USER\AppData\Local\Temp\ipykernel_16520\1614207678.py:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set

In [37]:
data.head()

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,PRGM_SALUD OCUPACIONAL,PRGM_SEGURIDAD Y SALUD EN EL TRABAJO,PRGM_SOCIOLOGIA,PRGM_TEOLOGIA,PRGM_TERAPIA OCUPACIONAL,PRGM_TERAPIA RESPIRATORIA,PRGM_TRABAJO SOCIAL,PRGM_ZOOTECNIA,ANIO,PERIODO
ID,,,,,,,,,,,,,,,,,,,,,
904256,6,1,3,1,5,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
645256,4,0,3,0,6,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2021,2
308367,4,4,3,1,4,1,0,0,0,0,...,False,False,False,False,False,False,False,False,2020,3
470353,5,0,4,1,-1,1,0,0,0,1,...,False,False,False,False,False,False,False,False,2019,5
989032,4,3,3,1,2,1,1,0,0,1,...,False,False,False,False,False,False,False,False,2021,2


In [38]:
#división entrenamiento y testeo
from sklearn.model_selection import train_test_split

X = data.drop(columns=["RENDIMIENTO_GLOBAL"])
y = data["RENDIMIENTO_GLOBAL"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # mantiene la proporción de clases
)

In [25]:
#librerias del modelo
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report


A continuación la prueba de varios arboles de decisión modificando parámetros

In [7]:
#modelo 1

modelo = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    multi_class='auto'
)

# Entrenamiento
modelo.fit(X_train, y_train)

# Predicción
y_pred = modelo.predict(X_test)

# Métricas
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", acc)
print("F1 Score:", f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.3742527075812274
F1 Score: 0.34889052272127613

Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.53      0.44     51896
           1       0.31      0.22      0.26     51682
           2       0.29      0.14      0.19     51486
           3       0.44      0.60      0.51     52686

    accuracy                           0.37    207750
   macro avg       0.35      0.37      0.35    207750
weighted avg       0.35      0.37      0.35    207750



In [9]:
#modelo 2

modelo = LogisticRegression(
    n_jobs=-1
)

modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", acc)
print("F1 Score:", f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.3558315282791817
F1 Score: 0.32809602234818064

Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.51      0.41     51896
           1       0.30      0.21      0.25     51682
           2       0.29      0.12      0.17     51486
           3       0.41      0.58      0.48     52686

    accuracy                           0.36    207750
   macro avg       0.34      0.35      0.33    207750
weighted avg       0.34      0.36      0.33    207750



No tiene mucho sentido seguir mirando la regresión logistica, con los datos esta estructura no dará buenos resultados